In [ ]:
from main import *

params, signals, models = main()
print("Everything has gone right.")

In [1]:
from core.lib.pyRequirment import *

In [29]:
a = np.arange(4)
a[2] = 0
a

array([0, 1, 0, 3])

In [19]:
np.add.reduce(np.double(-a), axis=(0,1))

-6.0

In [30]:
a[3] = np.nan

ValueError: cannot convert float NaN to integer

In [28]:
a[a == 0]

array([0, 0])